In [1]:
%%writefile main.cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

#ifndef CUDA_CHKERR
#define CUDA_CHKERR(err)                                          \
  do {                                                            \
    cudaError_t cuda_error = (err);                               \
    if (cuda_error != cudaSuccess) {                              \
      fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__,      \
              __LINE__, cudaGetErrorString(cuda_error));          \
      exit(EXIT_FAILURE);                                         \
    }                                                             \
  } while (0)
#endif

__global__ void helloFromGPU()
{
  printf("Hello World-Thread: %d\n", threadIdx.x);
}

int main(void)
{
  helloFromGPU<<<1, 16>>>();
  cudaDeviceSynchronize();
  CUDA_CHKERR(cudaGetLastError());
  return 0;
}

Writing main.cu


In [2]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [3]:
!./main.ex

Hello World-Thread: 0
Hello World-Thread: 1
Hello World-Thread: 2
Hello World-Thread: 3
Hello World-Thread: 4
Hello World-Thread: 5
Hello World-Thread: 6
Hello World-Thread: 7
Hello World-Thread: 8
Hello World-Thread: 9
Hello World-Thread: 10
Hello World-Thread: 11
Hello World-Thread: 12
Hello World-Thread: 13
Hello World-Thread: 14
Hello World-Thread: 15


In [4]:
%%writefile main.cu
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

#ifndef CUDA_CHKERR
#define CUDA_CHKERR(err)                                          \
  do {                                                            \
    cudaError_t cuda_error = (err);                               \
    if (cuda_error != cudaSuccess) {                              \
      fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__,      \
              __LINE__, cudaGetErrorString(cuda_error));          \
      exit(EXIT_FAILURE);                                         \
    }                                                             \
  } while (0)
#endif

__device__ double atomicAdd_CAS(double *address, double val)
{
    unsigned long long int* address_as_ull = (unsigned long long int*)address;
    unsigned long long int old = *address_as_ull, assumed;
    do {
        assumed = old;
        double old_double = __longlong_as_double(old);
        double new_double = old_double + val;
        unsigned long long int new_ull = __double_as_longlong(new_double);
        old = atomicCAS(address_as_ull, assumed, new_ull);
    } while (assumed != old);
    return __longlong_as_double(assumed);
}

__global__ void initializeArray(double *arr, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        arr[i] = (double)i;
    }
}

__global__ void atomicAddDemo(double *arr, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        atomicAdd_CAS(&arr[0], arr[i]);
    }
}

int main(void) {
    int N = 1024;
    double *arr_h, *arr_d;
    size_t size = N * sizeof(double);

    arr_h = (double*)malloc(size);
    CUDA_CHKERR(cudaMalloc(&arr_d, size));

    initializeArray<<<(N + 255) / 256, 256>>>(arr_d, N);
    CUDA_CHKERR(cudaGetLastError());
    CUDA_CHKERR(cudaDeviceSynchronize());


    atomicAddDemo<<<(N + 255) / 256, 256>>>(arr_d, N);
    CUDA_CHKERR(cudaGetLastError());
    CUDA_CHKERR(cudaDeviceSynchronize());

    CUDA_CHKERR(cudaMemcpy(arr_h, arr_d, size, cudaMemcpyDeviceToHost));

    printf("Sum: %f\n", arr_h[0]);

    free(arr_h);
    CUDA_CHKERR(cudaFree(arr_d));

    return 0;
}

Overwriting main.cu


In [5]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [6]:
!./main.ex

Sum: 523776.000000


In [7]:
%%writefile main.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

// I did not include error check. recommend to add!

#define BLOCK_SIZE 256

template <typename T>
__global__ void reduceSumV1(T *g_idata, T *g_odata, unsigned int n) {
    __shared__ T sdata[2*BLOCK_SIZE];

    unsigned int tid = threadIdx.x;
    unsigned int start = 2 * blockIdx.x * blockDim.x;

    sdata[tid] = (start + tid < n) ? g_idata[start + tid] : 0;
    sdata[tid+BLOCK_SIZE] = (start + tid + BLOCK_SIZE < n) ? g_idata[start + tid + BLOCK_SIZE] : 0;
    __syncthreads();

    for (unsigned int s = 1 ; s <= blockDim.x ; s *= 2) {
        if (tid % s == 0) {
            sdata[2*tid] += sdata[2*tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        g_odata[blockIdx.x] = sdata[0];
    }
}

template <typename T>
__global__ void reduceSumV2(T *g_idata, T *g_odata, unsigned int n) {
    __shared__ T sdata[2*BLOCK_SIZE];

    unsigned int tid = threadIdx.x;
    unsigned int start = 2 * blockIdx.x * blockDim.x;

    sdata[tid] = (start + tid < n) ? g_idata[start + tid] : 0;
    sdata[tid+BLOCK_SIZE] = (start + tid + BLOCK_SIZE < n) ? g_idata[start + tid + BLOCK_SIZE] : 0;
    __syncthreads();

    for (unsigned int s = blockDim.x; s > 0; s /= 2) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        g_odata[blockIdx.x] = sdata[0];
    }
}

template <typename T>
T cpuReduceSum(const std::vector<T>& data) {
    T sum = 0;
    for (const auto& val : data) {
        sum += val;
    }
    return sum;
}

int main() {
    unsigned int n = 1000000;
    std::vector<float> h_data(n);

    for (unsigned int i = 0; i < n; ++i) {
        h_data[i] = static_cast<float>(rand() % 10);
    }

    float *d_idata, *d_odata;
    cudaMalloc((void **)&d_idata, n * sizeof(float));

    unsigned int numBlocks = (n + 2 * BLOCK_SIZE - 1) / (2*BLOCK_SIZE);
    cudaMalloc((void **)&d_odata, numBlocks * sizeof(float));

    float *d_idata_v2, *d_odata_v2;
    cudaMalloc((void **)&d_idata_v2, n * sizeof(float));
    cudaMalloc((void **)&d_odata_v2, numBlocks * sizeof(float));

    cudaMemcpy(d_idata, h_data.data(), n * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_idata_v2, h_data.data(), n * sizeof(float), cudaMemcpyHostToDevice);

    reduceSumV1<float><<<numBlocks, BLOCK_SIZE>>>(d_idata, d_odata, n);
    reduceSumV2<float><<<numBlocks, BLOCK_SIZE>>>(d_idata_v2, d_odata_v2, n);
    cudaDeviceSynchronize();

    std::vector<float> partialSumsV1(numBlocks);
    cudaMemcpy(partialSumsV1.data(), d_odata, numBlocks * sizeof(float), cudaMemcpyDeviceToHost);

    std::vector<float> partialSumsV2(numBlocks);
    cudaMemcpy(partialSumsV2.data(), d_odata_v2, numBlocks * sizeof(float), cudaMemcpyDeviceToHost);

    float finalSumV1 = cpuReduceSum(partialSumsV1);
    float finalSumV2 = cpuReduceSum(partialSumsV2);

    float cpuSum = cpuReduceSum(h_data);
    std::cout << "CUDA Reduction V1 Sum: " << finalSumV1 << std::endl;
    std::cout << "CUDA Reduction V2 Sum: " << finalSumV2 << std::endl;
    std::cout << "CPU Sum: " << cpuSum << std::endl;
    if (std::abs(finalSumV1 - cpuSum) < 1e-5 && std::abs(finalSumV2 - cpuSum) < 1e-5 ) {
        std::cout << "Results match!" << std::endl;
    } else {
        std::cout << "Error: Results do not match!" << std::endl;
    }

    cudaFree(d_idata);
    cudaFree(d_odata);

    return 0;
}

Overwriting main.cu


In [8]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [9]:
!./main.ex

CUDA Reduction V1 Sum: 4.50273e+06
CUDA Reduction V2 Sum: 4.50273e+06
CPU Sum: 4.50273e+06
Results match!


In [10]:
%%writefile main.cu
#include <iostream>
#include <vector>
#include <cuda_runtime.h>
#include <algorithm>

// I did not include error check. recommend to add!

#define BLOCK_SIZE 256

// Note: this code is for illustrative purpose. Deos not work for more than one block.
template <typename T>
__global__ void prefixSumKernelMultiple(T *g_idata, T *g_odata, int n) {
    __shared__ T sdata[2 * BLOCK_SIZE];

    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * (BLOCK_SIZE * 2) + tid;

    T val0 = (i < n) ? g_idata[i] : 0;
    T val1 = (i + BLOCK_SIZE < n) ? g_idata[i + BLOCK_SIZE] : 0;

    sdata[tid] = val0;
    sdata[tid + BLOCK_SIZE] = val1;
    __syncthreads();

    for (unsigned int stride = 1; stride < 2 * BLOCK_SIZE; stride *= 2) {
        int index = (tid + 1) * stride * 2 - 1;
        if (index < 2 * BLOCK_SIZE) {
            sdata[index] += sdata[index - stride];
        }
        __syncthreads();
    }

     for (int stride = BLOCK_SIZE; stride > 0; stride /= 2) {
        int index = (tid + 1) * stride * 2 - 1;
        if (index + stride < 2 * BLOCK_SIZE) {
            sdata[index + stride] += sdata[index];
        }
        __syncthreads();
    }

    if (i < n) {
        g_odata[i] = sdata[tid];
    }
    if (i + BLOCK_SIZE < n) {
        g_odata[i + BLOCK_SIZE] = sdata[tid + BLOCK_SIZE];
    }
}

template <typename T>
void cpuPrefixSum(std::vector<T>& data) {
    for (size_t i = 1; i < data.size(); ++i) {
        data[i] += data[i - 1];
    }
}

int main() {
    int n = 2*BLOCK_SIZE; // only one block for demo!
    std::vector<int> h_data(n);

    for (int i = 0; i < n; ++i) {
        h_data[i] = i + 1;
    }

    std::vector<int> h_cpuResult = h_data;
    cpuPrefixSum(h_cpuResult);

    int *d_idata, *d_odata;
    cudaMalloc((void **)&d_idata, n * sizeof(int));
    cudaMalloc((void **)&d_odata, n * sizeof(int));

    cudaMemcpy(d_idata, h_data.data(), n * sizeof(int), cudaMemcpyHostToDevice);

    int numBlocks = (n + (BLOCK_SIZE * 2) - 1) / (BLOCK_SIZE * 2);

    prefixSumKernelMultiple<int><<<numBlocks, BLOCK_SIZE>>>(d_idata, d_odata, n);
    cudaDeviceSynchronize();

    std::vector<int> h_gpuResult(n);
    cudaMemcpy(h_gpuResult.data(), d_odata, n * sizeof(int), cudaMemcpyDeviceToHost);

    bool success = true;
    for (int i = 0; i < n; ++i) {
        if (h_gpuResult[i] != h_cpuResult[i]) {
             std::cout << "Error at index " << i << ": GPU = " << h_gpuResult[i]
                      << ", CPU = " << h_cpuResult[i] << std::endl;
            success = false;
            break;
        }
    }

    if(success)
    {
        std::cout << "Prefix Sum calculation successful!" << std::endl;
    }

    cudaFree(d_idata);
    cudaFree(d_odata);

    return 0;
}

Overwriting main.cu


In [11]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [12]:
!./main.ex

Prefix Sum calculation successful!


In [13]:
%%writefile main.cu
#include <iostream>
#include <string>
#include <vector>
#include <cuda_runtime.h>

// I did not include error check. recommend to add!

__global__ void countCharactersKernel(const char* text, int* counts, int textSize) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < textSize) {
        char c = text[tid];

        if (c >= 'A' && c <= 'Z') {
            c += 32; // Convert to lowercase
        }

        if (c >= 'a' && c <= 'z') {
            atomicAdd(&counts[c - 'a'], 1); // Increment the count for the corresponding letter
        }
    }
}

int main() {
    std::string text = "High Performance Computing: Tools and Applications";
    int textSize = text.length();

    std::vector<int> h_counts(26, 0); // 26 letters (a-z)

    char* d_text;
    int* d_counts;
    cudaMalloc(&d_text, textSize * sizeof(char));
    cudaMalloc(&d_counts, 26 * sizeof(int));

    cudaMemcpy(d_text, text.c_str(), textSize * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemset(d_counts, 0, 26 * sizeof(int));

    int blockSize = 256;
    int numBlocks = (textSize + blockSize - 1) / blockSize;

    countCharactersKernel<<<numBlocks, blockSize>>>(d_text, d_counts, textSize);
    cudaDeviceSynchronize();

    cudaMemcpy(h_counts.data(), d_counts, 26 * sizeof(int), cudaMemcpyDeviceToHost);

    std::cout << "Character Counts:" << std::endl;
    for (int i = 0; i < 26; i++) {
        if (h_counts[i] > 0) { // Only print letters that actually appear
            std::cout << char('a' + i) << ": " << h_counts[i] << std::endl;
        }
    }

    cudaFree(d_text);
    cudaFree(d_counts);

    return 0;
}

Overwriting main.cu


In [14]:
!nvcc -arch=sm_70 main.cu -o main.ex

In [15]:
!./main.ex

Character Counts:
a: 4
c: 3
d: 1
e: 2
f: 1
g: 2
h: 2
i: 4
l: 2
m: 2
n: 4
o: 5
p: 4
r: 2
s: 2
t: 3
u: 1
